<a href="https://colab.research.google.com/github/mosina003/RESUME-CREATOR/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install nltk

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import json
import pickle



import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [6]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']

data_file=open('/content/data.json').read()
intents=json.loads(data_file)

In [8]:
for intent in intents["intents"]:
    for pattern in intent['patterns']:
        #take each word and tokenize
        w=nltk.word_tokenize(pattern)
        words.extend(w)

        #adding documents
        documents.append((w,intent['tag']))

        #adding class to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))

In [10]:
print(len(documents),"Documents: ",documents)
print(len(classes),"classes: ",classes)
print(len(words),"Unique Lemmatized words",words)

8 Documents:  [(['Hi', 'there'], 'greetings'), (['How', 'are', 'you', '?'], 'greetings'), (['Hello'], 'greetings'), (['What', 'is', 'supervised', 'learning', '?'], 'suplearn'), (['Supervised', 'learning'], 'suplearn'), (['define', 'supervised', 'learning'], 'suplearn'), (['Give', 'an', 'example', 'of', 'classification'], 'Example_classification'), (['example', 'of', 'classification'], 'Example_classification')]
3 classes:  ['Example_classification', 'greetings', 'suplearn']
16 Unique Lemmatized words ['an', 'are', 'classification', 'define', 'example', 'give', 'hello', 'hi', 'how', 'is', 'learning', 'of', 'supervised', 'there', 'what', 'you']


In [11]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

In [12]:
#initializing training data
training=[]
output_empty=[0]*len(classes)
for doc in documents:
    #initializing the bag of words
    bag=[]
    #list of tokenized words for pattern
    pattern_words=doc[0]
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1

    training.append([bag,output_row])

print("training:",training)

random.shuffle(training)
training = np.array(training, dtype=object)


train_x=list(training[:,0])
train_y=list(training[:,1])
print("Training data created")
print("train_x:" ,train_x)
print("train_y:" ,train_y)

training: [[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0]], [[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 0, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1]], [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1]], [[1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0]], [[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0]]]
Training data created
train_x: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]]
train_y: [[0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0],

In [13]:
model=Sequential()
#input layer
model.add(Dense(256, input_shape=(len(train_x[0]),)))
model.add(Activation('relu'))

#hidden layer1
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#hidden layer2
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(len(train_y[0])))
model.add(Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

hist=model.fit(np.array(train_x), np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)

print('model created')

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6167 - loss: 1.0683
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4667 - loss: 1.0708 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4500 - loss: 1.0804
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3167 - loss: 1.0520 
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9167 - loss: 0.9319 
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.8163
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.8282 
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8500 - loss: 0.8189 
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8500 - loss: 0.7032 
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8500 - loss: 0.6814
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8500 - loss: 0.6317 
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8500 - los

model created
